In [ ]:
!git clone  https://github.com/daniela-figueroa/CHEMENG177
import pandas as pd
import numpy as np
import copy

Cloning into 'CHEMENG177'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 18 (delta 6), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (18/18), 413.73 KiB | 5.59 MiB/s, done.
Resolving deltas: 100% (6/6), done.


In [ ]:
%cd CHEMENG177

/content/CHEMENG177


In [ ]:
ls

'CALiSol-23 Dataset.csv'   CALiSol-23.ipynb   README.md   ridge.ipynb


# Initial Setup

In [ ]:
dataset = pd.read_csv("CALiSol-23 Dataset.csv")
print(dataset.shape)


(13825, 45)


In [ ]:
#remove rows with no k value
dataset = dataset[(dataset['k'].isnull() == 0)]
print(dataset.shape)
#523 rows with NaN values for k (conductivity)

(13302, 45)


In [ ]:
print(dataset.columns)

Index(['doi', 'k', 'T', 'c', 'salt', 'c units', 'solvent ratio type', 'EC',
       'PC', 'DMC', 'EMC', 'DEC', 'DME', 'DMSO', 'AN', 'MOEMC', 'TFP', 'EA',
       'MA', 'FEC', 'DOL', '2-MeTHF', 'DMM', 'Freon 11', 'Methylene chloride',
       'THF', 'Toluene', 'Sulfolane', '2-Glyme', '3-Glyme', '4-Glyme',
       '3-Me-2-Oxazolidinone', '3-MeSulfolane', 'Ethyldiglyme', 'DMF',
       'Ethylbenzene', 'Ethylmonoglyme', 'Benzene', 'g-Butyrolactone',
       'Cumene', 'Propylsulfone', 'Pseudocumeme', 'TEOS', 'm-Xylene',
       'o-Xylene'],
      dtype='object')


Convert all numbers into floats

In [ ]:
#converts all numbers into floats to avoid future issues
numeric_cols = dataset.select_dtypes(include=['number']).columns  # Or: df.select_dtypes(exclude=['object']).columns
dataset[numeric_cols] = dataset[numeric_cols].astype(float)

type(dataset.iloc[71, 8])
dataset.iloc[:, 7:]

,EC,PC,DMC,EMC,DEC,DME,DMSO,AN,MOEMC,TFP,...,Ethylbenzene,Ethylmonoglyme,Benzene,g-Butyrolactone,Cumene,Propylsulfone,Pseudocumeme,TEOS,m-Xylene,o-Xylene
0,0.330,0.000,0.33,0.0,0.33,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.330,0.000,0.33,0.0,0.33,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.330,0.000,0.33,0.0,0.33,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.330,0.000,0.33,0.0,0.33,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.330,0.000,0.33,0.0,0.33,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13820,0.500,0.500,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13821,0.000,0.000,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13822,0.125,0.125,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13823,0.500,0.500,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Moles to Liters Function

In [ ]:
def transform_mole_to_liter(df, molar_weights, densities):
    '''
        Function that transforms a df with molar weights in one kg, into how much
        volume is occupied in the mixture. This assumes that the volume of mixed
        liquids depends on the average of their density

        Returns the volume of each compositon in [l]
    '''
    copied_df = copy.copy(df)

    # Transform moles into g of
    for column in copied_df.columns:
        if column in molar_weights:
            copied_df[column] *= molar_weights[column]

    # Now each column is divided by the density
    for column in copied_df.columns:
        if column in densities:
            copied_df[column] *= 1 / (densities[column] * 1000)
    total_volumes = copied_df.iloc[:,7:].sum(axis=1)
    return total_volumes

Dictionary Setup

In [ ]:
molar_weights = {'EC': 88.06,
       'PC' : 102.08, 'DMC' : 90.08,'EMC' : 104.10, 'DEC' : 118.132,
        'DME' : 90.12, 'DMSO' : 78.13, 'AN': 41.05, 'MOEMC': 134.13, 'TFP': 344.07, 'EA' : 88.10,
       'MA': 74.08, 'FEC': 106.05, 'DOL': 74.08, '2-MeTHF' : 86.13, 'DMM': 162.2, 'Freon 11' : 137.36,
       'Methylene chloride' : 84.93,
       'THF' : 72.10, 'Toluene' : 92.14, 'Sulfolane' : 120.17, '2-Glyme' : 134.17, '3-Glyme' : 178.22,
                 '4-Glyme' : 222.28,
       '3-Me-2-Oxazolidinone' : 101.10, '3-MeSulfolane' : 134.20, 'Ethyldiglyme': 134.17, 'DMF' : 73.09,
       'Ethylbenzene' : 106.17, 'Ethylmonoglyme' : 76.10, 'Benzene' : 78.11, 'g-Butyrolactone' : 86.09,
       'Cumene' : 120.19, 'Propylsulfone' : 150.24, 'Pseudocumeme' : 120.19, 'TEOS' : 208.33, 'm-Xylene' : 106.17,
       'o-Xylene' : 106.16} # Molar weights of different organic solvents  [g / mol]
molar_weights_salts = {'LiPF6' : 151.91, 'LiBF4': 93.75,
                      'LiFSI': 187.7, 'LiTDI' : 192.1, 'LiPDI' : 242.1, 'LiTFSI' : 287.07, 'LiClO4' : 160.44, 'LiAsF6' : 195.9,
 'LiBOB' : 193.79, 'LiCF3SO3' : 156.01, 'LiBPFPB' : 193.8, 'LiBMB': 221.85, 'LiN(CF3SO2)2' : 287.07} # Molar weights of salts in [g / mol]. Note that LiClO4 exists in hydrous and anyhydrous form
# Note that the weight of LiBMB was calculated theoretically
names_salts = {'LiPF6' : "Lithium hexafluorophosphate", 'LiBF4': "Lithium tetrafluoroborate",
                      'LiFSI': "Lithium Bis(fluorosulfonyl)imide", 'LiTDI' : "lithium 2-trifluoromethyl-4,5-dicyanoimidazole",
               'LiPDI' : "lithium 4,5-dicyano-2-(pentafluoroethyl)imidazolide", 'LiTFSI': "Lithium bis(trifluoromethanesulfonyl)imide",
               'LiClO4' : "Lithium perchlorate", 'LiAsF6' : "Lithium hexafluoroarsenate(V)",
 'LiBOB' : "Lithium bis(oxalato)borate", 'LiCF3SO3' : "Lithium triflate", 'LiBPFPB' : "Lithium bis(oxalate)borate",
               'LiBMB' : "lithium bis(malonato)borate", 'LiN(CF3SO2)2' : "lithium bis(trifluoromethanesulfonimide)"}

names_solvents = {'EC' : "Ethylene carbonate",
       'PC' : "Propylene carbonate", 'DMC' : "Dimethyl carbonate", 'EMC' : "Ethyl Methyl Carbonate",
         'DEC' : "​Diethyl carbonate", 'DME' : "Dimethoxyethane", 'DMSO' : "Dimethyl sulfoxide", 'AN' : "Acetonitrile", 'MOEMC' : "2-Methoxyethyl (methyl) carbonate",
         'TFP' : "Tris(2,2,2-trifluoroethyl) phosphate", 'EA' : "Ethyl acetate",
       'MA' : "Methyl acetate", 'FEC' : "Fluoroethylene carbonate", 'DOL': "Dioxolane"
         , '2-MeTHF': "2-Methyltetrahydrofuran", 'DMM' : "Dipropylene glycol dimethyl ether",
         'Freon 11' : "Trichlorofluoromethane", 'Methylene chloride' : 'Methylene chloride',
       'THF' : "Tetrahydrofuran", 'Toluene' : "Toluene", 'Sulfolane' : "Sulfolane",
         '2-Glyme': "Diglyme", '3-Glyme': "Triglyme", '4-Glyme': "tetraglyme",
       '3-Me-2-Oxazolidinone' : "3-Me-2-Oxazolidinone", '3-MeSulfolane' : "3-Methylsulfolane",
         'Ethyldiglyme' : "2-(2-Ethoxyethoxy)ethanol", 'DMF' : "Dimethylformamide",
       'Ethylbenzene': 'Ethylbenzene', 'Ethylmonoglyme': "ethylene glycol monomethyl", 'Benzene' : "Benzene", 'g-Butyrolactone' : "gamma-Butyrolactone",
       'Cumene' : "Cumene", 'Propylsulfone' : 'Propylsulfone', 'Pseudocumeme' : "1,2,4-Trimethylbenzene", 'TEOS' : "Tetraethyl orthosilicate", 'm-Xylene' : 'm-Xylene',
       'o-Xylene' : 'o-Xylene'
}

densities = {'EC': 1.3210,
       'PC' : 1.205, 'DMC' : 1.07, 'EMC' : 0.902, 'DEC' : 0.975,
        'DME' : 0.86, 'DMSO' : 1.1004, 'AN': 0.786, 'MOEMC': 1.5, 'TFP': 1.487, 'EA' : 0.902,
       'MA': 0.932, 'FEC': 1.454, 'DOL': 1.06, '2-MeTHF' : 0.854, 'DMM': 0.902, 'Freon 11': 1.49, 'Methylene chloride': 1.33,
       'THF': 0.888, 'Toluene' : 0.867, 'Sulfolane' : 1.26, '2-Glyme': 0.937, '3-Glyme': 0.986, '4-Glyme': 1.009,
       '3-Me-2-Oxazolidinone': 1.17, '3-MeSulfolane': 1.20, 'Ethyldiglyme' : 0.937, 'DMF': 0.944,
       'Ethylbenzene': 0.866, 'Ethylmonoglyme': 0.965, 'Benzene': 0.876, 'g-Butyrolactone' : 1.13,
       'Cumene': 0.862, 'Propylsulfone' : 1.109, 'Pseudocumeme': 0.876, 'TEOS': 0.940, 'm-Xylene' : 0.860,
       'o-Xylene': 0.87596} # g / cm3 = g / ml at 25 C

#NOTE: The salt density values for LiPDI is approximated based off of LiTDI, and the salt desnity values for LiBPFPB and LiBMB are approximated off of LiBOB.
densities_salts = {'LiPF6' : 2.84, 'LiBF4': 0.852,
                      'LiFSI': 1.052, 'LiTDI' : 2.2, 'LiPDI' : 2.1, 'LiTFSI' : 1.33, 'LiClO4' : 2.42, 'LiAsF6' : 2.32,
 'LiBOB' : 2.021, 'LiCF3SO3' : 1.9, 'LiBPFPB' : 2.10, 'LiBMB': 2.00, 'LiN(CF3SO2)2' : 1.33}


# Transform All Solvent Ratios into Moles

Assume 1 kg of solvent.
Assume there is no density gradient in the solvent.

In [ ]:
mass_solvent = 1 # 1 kg
copied_dataset = copy.copy(dataset)

# Extracting all columns, where the solvent ratios are given by their weights, molarity and volumes
solvent_ratio_w = copied_dataset.loc[copied_dataset['solvent ratio type'] == 'w']
solvent_ratio_m_o = copied_dataset.loc[copied_dataset['solvent ratio type'] == 'mol'] #df column containing moles of solvent component after calculations on mol ratios
solvent_ratio_m = copied_dataset.loc[copied_dataset['solvent ratio type'] == 'mol'] # df column containing g / mol sum of solvent component after calcs on mol ratios
solvent_ratio_v = copied_dataset.loc[copied_dataset['solvent ratio type'] == 'v'] # df column containing g / l of solvent component after calculations on mol ratios
solvent_ratio_v_o = copied_dataset.loc[copied_dataset['solvent ratio type'] == 'v'] # df column containing moles of solvent component after calculations on mol ratios

## Weight Ratios into Moles

In [ ]:
# Transforming solvent weight ratios to how many moles in one kg of solvent
# (w = mass / mass sum) * (1000 g / 1 kg) * (1 / MW) * 1 kg
# dimensional analysis: (g / g1 + g2...) * (1000 g / 1 kg ) = (g / kg1 + kg2...) * (mol / g) = (mol / kg1 + kg2...) * (1 kg solv = kg1 + kg2...) = mol
for column in solvent_ratio_w.columns:
    if column in molar_weights:
        solvent_ratio_w.loc[:, column] *= (1000 / molar_weights[column]) * mass_solvent # Result is moles of solvent

In [ ]:
copied_dataset.loc[copied_dataset['solvent ratio type'] == 'w'] = solvent_ratio_w
copied_dataset.loc[copied_dataset['solvent ratio type'] == 'w']


,doi,k,T,c,salt,c units,solvent ratio type,EC,PC,DMC,...,Ethylbenzene,Ethylmonoglyme,Benzene,g-Butyrolactone,Cumene,Propylsulfone,Pseudocumeme,TEOS,m-Xylene,o-Xylene
0,10.1007/s10008-002-0300-9,0.160643,333.235034,1.0,LiPF6,mol/kg,w,3.747445,0.0,3.66341,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10.1007/s10008-002-0300-9,0.150856,322.764924,1.0,LiPF6,mol/kg,w,3.747445,0.0,3.66341,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10.1007/s10008-002-0300-9,0.141666,313.801722,1.0,LiPF6,mol/kg,w,3.747445,0.0,3.66341,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,10.1007/s10008-002-0300-9,0.132277,303.681806,1.0,LiPF6,mol/kg,w,3.747445,0.0,3.66341,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,10.1007/s10008-002-0300-9,0.119348,293.049793,1.0,LiPF6,mol/kg,w,3.747445,0.0,3.66341,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12190,10.1149/2.0981803jes,13.930000,313.150000,2.0,LiPF6,mol/kg,w,3.406768,0.0,7.77087,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12191,10.1149/2.0981803jes,6.360000,313.150000,2.0,LiPF6,mol/kg,w,0.000000,0.0,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12192,10.1149/2.0981803jes,8.690000,313.150000,2.0,LiPF6,mol/kg,w,1.135589,0.0,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12193,10.1149/2.0981803jes,9.070000,313.150000,2.0,LiPF6,mol/kg,w,2.271179,0.0,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
solvent_ratio_w

,doi,k,T,c,salt,c units,solvent ratio type,EC,PC,DMC,...,Ethylbenzene,Ethylmonoglyme,Benzene,g-Butyrolactone,Cumene,Propylsulfone,Pseudocumeme,TEOS,m-Xylene,o-Xylene
0,10.1007/s10008-002-0300-9,0.160643,333.235034,1.0,LiPF6,mol/kg,w,3.747445,0.0,3.66341,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0
1,10.1007/s10008-002-0300-9,0.150856,322.764924,1.0,LiPF6,mol/kg,w,3.747445,0.0,3.66341,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0
2,10.1007/s10008-002-0300-9,0.141666,313.801722,1.0,LiPF6,mol/kg,w,3.747445,0.0,3.66341,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0
3,10.1007/s10008-002-0300-9,0.132277,303.681806,1.0,LiPF6,mol/kg,w,3.747445,0.0,3.66341,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0
4,10.1007/s10008-002-0300-9,0.119348,293.049793,1.0,LiPF6,mol/kg,w,3.747445,0.0,3.66341,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12190,10.1149/2.0981803jes,13.930000,313.150000,2.0,LiPF6,mol/kg,w,3.406768,0.0,7.77087,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0
12191,10.1149/2.0981803jes,6.360000,313.150000,2.0,LiPF6,mol/kg,w,0.000000,0.0,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0
12192,10.1149/2.0981803jes,8.690000,313.150000,2.0,LiPF6,mol/kg,w,1.135589,0.0,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0
12193,10.1149/2.0981803jes,9.070000,313.150000,2.0,LiPF6,mol/kg,w,2.271179,0.0,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0


## Mol Ratios into Moles

In [ ]:
# Transforming solvent molar ratios to how many moles in one kg of solvent
# (m = mol / mol sum) * (MW = mass / mol ) = mass/mol sum. sum of (mass/mol sum)'s yields (mass sum / mol sum).
 #(mol / mol sum) * 1/(mass sum / mol sum) * (1000 g / 1 kg) = (mol / mass sum [kg])* 1 kg of solvent = moles

# dimensional analysis: (mol / mol1 + mol2...) * (g / mol) = ( g / mol1 + mol2...).
 #(1000 g / 1 kg) * ( mol1 + mol2... / g1 + g2...) = (mol1 + mol2... / kg1 + kg2...) * (mol / mol1 + mol2...) = (mol / kg1 + kg2...) * (1 kg of solvent = kg1 + kg2...) = mol
for column in solvent_ratio_m.columns:
    if column in molar_weights:
        solvent_ratio_m.loc[:, column] *= molar_weights[column]
row_sums = 1000 / solvent_ratio_m.iloc[:, 7:].sum(axis=1).values # Sum of molar fraction times the molar weight

#  each row by 1000 divided by the sum of that row
normalized_df_m = solvent_ratio_m_o.iloc[:, 7:].mul(row_sums, axis=0)
solvent_ratio_m_o.iloc[:, 7:] = normalized_df_m * mass_solvent # Now in each row for the solvents, we have the number of moles per kg of solvent

In [ ]:
copied_dataset.loc[copied_dataset['solvent ratio type'] == 'mol'] = solvent_ratio_m_o
copied_dataset.loc[copied_dataset['solvent ratio type'] == 'mol']

,doi,k,T,c,salt,c units,solvent ratio type,EC,PC,DMC,...,Ethylbenzene,Ethylmonoglyme,Benzene,g-Butyrolactone,Cumene,Propylsulfone,Pseudocumeme,TEOS,m-Xylene,o-Xylene
130,10.1016/j.jpowsour.2011.07.071,5.037117,260.912465,1.0,LiPF6,mol/l,mol,5.613562,0.0,5.613562,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
131,10.1016/j.jpowsour.2011.07.071,5.194092,261.679555,1.0,LiPF6,mol/l,mol,5.613562,0.0,5.613562,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
132,10.1016/j.jpowsour.2011.07.071,5.247448,262.137334,1.0,LiPF6,mol/l,mol,5.613562,0.0,5.613562,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
133,10.1016/j.jpowsour.2011.07.071,5.353387,262.901330,1.0,LiPF6,mol/l,mol,5.613562,0.0,5.613562,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
134,10.1016/j.jpowsour.2011.07.071,5.457006,263.210640,1.0,LiPF6,mol/l,mol,5.613562,0.0,5.613562,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5649,10.1149/1.1527939,2.199510,228.434917,1.2,LiBOB,mol/l,mol,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5650,10.1149/1.1527939,1.728628,223.393383,1.2,LiBOB,mol/l,mol,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5651,10.1149/1.1527939,1.249148,219.195667,1.2,LiBOB,mol/l,mol,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5652,10.1149/1.1527939,0.891675,213.679275,1.2,LiBOB,mol/l,mol,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Volume Ratios into Moles

In [ ]:
# Transforming solvent volume ratios to how many moles in one kg of solvent
# (v = vol / vol sum) * (density = mass / vol) = mass/vol sum. sum of (mass / vol sum)'s yields (mass sum / vol sum).
# (vol / vol sum) * 1/(mass sum / vol sum) * (1000 g / 1 kg) = (vol / mass sum [kg]).
# (vol / mass sum [kg]) * (density = mass / vol) * 1/(MW = mol / mass) = (mol / mass sum [kg]) for solvent mass of one kg =  mol

# dimensional analysis: (ml / ml1 + ml2...) * (g / ml) = (g / ml1 + ml2...).
# (1000g / 1 kg) * (ml1 + ml2... / g1 + g2...) = (ml1 + ml2... / kg1 + kg2...).
# (ml1 + ml2... / kg1 + kg2...) * (ml / ml1 + ml2...)  = (ml / kg1 + kg2...) * (g / ml) = (g / kg1 + kg2...) * (mol / g). for kg1 + kg2... = 1 kg : mol
for column in solvent_ratio_v.columns:
    if column in molar_weights:
        solvent_ratio_v.loc[:, column] *= densities[column] # (ml / ml1 + ml2...) * (g / ml) = (g / ml1 + ml2...)
row_sums = 1000 / solvent_ratio_v.iloc[:, 7:].sum(axis=1).values # (1000g / 1 kg) * (ml1 + ml2... / g1 + g2...) = (ml1 + ml2... / kg1 + kg2...) Sum of vol fraction times the densities and converted to ml / kg.

#  each row by 1000 divided by the sum of that row
normalized_df_v = solvent_ratio_v_o.iloc[:, 7:].mul(row_sums, axis=0) # (ml1 + ml2... / kg1 + kg2...) * (ml / ml1 + ml2...)  = (ml / kg1 + kg2...)
solvent_ratio_v_o.iloc[:, 7:] = normalized_df_v # Now in each row for the solvents components, we have the ml per kg of solvent
for column in solvent_ratio_v_o.columns:
    if column in molar_weights:
        solvent_ratio_v_o.loc[:, column] *= densities[column] # (ml / kg1 + kg2...) * (g / ml) = (g / kg1 + kg2...)
        solvent_ratio_v_o.loc[:, column] = (solvent_ratio_v_o.loc[:, column] / molar_weights[column]) * mass_solvent # (g / kg1 + kg2...) * (mol / g). for kg1 + kg2... = 1 kg : mol


In [ ]:
copied_dataset.loc[copied_dataset['solvent ratio type'] == 'v'] = solvent_ratio_v_o
copied_dataset.loc[copied_dataset['solvent ratio type'] == 'v']

,doi,k,T,c,salt,c units,solvent ratio type,EC,PC,DMC,...,Ethylbenzene,Ethylmonoglyme,Benzene,g-Butyrolactone,Cumene,Propylsulfone,Pseudocumeme,TEOS,m-Xylene,o-Xylene
31,10.1016/j.jpowsour.2010.12.040,15.583040,332.542042,1.0,LiFSI,mol/l,v,4.379041,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
32,10.1016/j.jpowsour.2010.12.040,13.928809,322.524407,1.0,LiFSI,mol/l,v,4.379041,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
33,10.1016/j.jpowsour.2010.12.040,12.286888,312.743178,1.0,LiFSI,mol/l,v,4.379041,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34,10.1016/j.jpowsour.2010.12.040,10.696357,302.713120,1.0,LiFSI,mol/l,v,4.379041,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35,10.1016/j.jpowsour.2010.12.040,9.009385,292.849497,1.0,LiFSI,mol/l,v,4.379041,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13820,10.1016/0378-7753(91)80004-H,8.510000,333.150000,1.0,LiBF4,mol/l,v,5.938692,4.673186,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13821,10.1016/0378-7753(91)80004-H,2.530000,333.150000,0.5,LiBF4,mol/l,v,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13822,10.1016/0378-7753(91)80004-H,4.290000,353.150000,1.0,LiBF4,mol/l,v,1.960933,1.543068,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13823,10.1016/0378-7753(91)80004-H,10.790000,353.150000,1.0,LiBF4,mol/l,v,5.938692,4.673186,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
copied_dataset

,doi,k,T,c,salt,c units,solvent ratio type,EC,PC,DMC,...,Ethylbenzene,Ethylmonoglyme,Benzene,g-Butyrolactone,Cumene,Propylsulfone,Pseudocumeme,TEOS,m-Xylene,o-Xylene
0,10.1007/s10008-002-0300-9,0.160643,333.235034,1.0,LiPF6,mol/kg,w,3.747445,0.000000,3.66341,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10.1007/s10008-002-0300-9,0.150856,322.764924,1.0,LiPF6,mol/kg,w,3.747445,0.000000,3.66341,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10.1007/s10008-002-0300-9,0.141666,313.801722,1.0,LiPF6,mol/kg,w,3.747445,0.000000,3.66341,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,10.1007/s10008-002-0300-9,0.132277,303.681806,1.0,LiPF6,mol/kg,w,3.747445,0.000000,3.66341,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,10.1007/s10008-002-0300-9,0.119348,293.049793,1.0,LiPF6,mol/kg,w,3.747445,0.000000,3.66341,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13820,10.1016/0378-7753(91)80004-H,8.510000,333.150000,1.0,LiBF4,mol/l,v,5.938692,4.673186,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13821,10.1016/0378-7753(91)80004-H,2.530000,333.150000,0.5,LiBF4,mol/l,v,0.000000,0.000000,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13822,10.1016/0378-7753(91)80004-H,4.290000,353.150000,1.0,LiBF4,mol/l,v,1.960933,1.543068,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13823,10.1016/0378-7753(91)80004-H,10.790000,353.150000,1.0,LiBF4,mol/l,v,5.938692,4.673186,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Transform salt concentration (c) into moles of salt

In [ ]:
dataset.loc[(dataset['salt'] == "LiPF6") & (dataset['EC'] > 0)]

,Unnamed: 0,doi,k,T,c,salt,c units,solvent ratio type,EC,PC,...,Ethylbenzene,Ethylmonoglyme,Benzene,g-Butyrolactone,Cumene,Propylsulfone,Pseudocumeme,TEOS,m-Xylene,o-Xylene
0,0,10.1007/s10008-002-0300-9,0.160643,333.235034,1.0,LiPF6,mol/kg,w,0.330,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0
1,1,10.1007/s10008-002-0300-9,0.150856,322.764924,1.0,LiPF6,mol/kg,w,0.330,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0
2,2,10.1007/s10008-002-0300-9,0.141666,313.801722,1.0,LiPF6,mol/kg,w,0.330,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0
3,3,10.1007/s10008-002-0300-9,0.132277,303.681806,1.0,LiPF6,mol/kg,w,0.330,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0
4,4,10.1007/s10008-002-0300-9,0.119348,293.049793,1.0,LiPF6,mol/kg,w,0.330,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13642,13642,10.1016/0378-7753(91)80004-H,12.800000,333.150000,1.0,LiPF6,mol/l,v,0.150,0.850,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0
13643,13643,10.1016/0378-7753(91)80004-H,14.630000,333.150000,1.0,LiPF6,mol/l,v,0.500,0.500,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0
13644,13644,10.1016/0378-7753(91)80004-H,16.220000,353.150000,1.0,LiPF6,mol/l,v,0.125,0.125,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0
13645,13645,10.1016/0378-7753(91)80004-H,16.880000,353.150000,1.0,LiPF6,mol/l,v,0.150,0.850,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0


In [ ]:
# Extracting molarity and molality subsets
salt_molality = copied_dataset.loc[copied_dataset['c units'] == 'mol/kg']
salt_molarity = copied_dataset.loc[copied_dataset['c units'] == 'mol/l']

## Molality conversion

In [ ]:
salt_molality

,doi,k,T,c,salt,c units,solvent ratio type,EC,PC,DMC,...,Ethylbenzene,Ethylmonoglyme,Benzene,g-Butyrolactone,Cumene,Propylsulfone,Pseudocumeme,TEOS,m-Xylene,o-Xylene
0,10.1007/s10008-002-0300-9,0.160643,333.235034,1.0,LiPF6,mol/kg,w,3.747445,0.0,3.66341,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10.1007/s10008-002-0300-9,0.150856,322.764924,1.0,LiPF6,mol/kg,w,3.747445,0.0,3.66341,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10.1007/s10008-002-0300-9,0.141666,313.801722,1.0,LiPF6,mol/kg,w,3.747445,0.0,3.66341,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,10.1007/s10008-002-0300-9,0.132277,303.681806,1.0,LiPF6,mol/kg,w,3.747445,0.0,3.66341,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,10.1007/s10008-002-0300-9,0.119348,293.049793,1.0,LiPF6,mol/kg,w,3.747445,0.0,3.66341,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12190,10.1149/2.0981803jes,13.930000,313.150000,2.0,LiPF6,mol/kg,w,3.406768,0.0,7.77087,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12191,10.1149/2.0981803jes,6.360000,313.150000,2.0,LiPF6,mol/kg,w,0.000000,0.0,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12192,10.1149/2.0981803jes,8.690000,313.150000,2.0,LiPF6,mol/kg,w,1.135589,0.0,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12193,10.1149/2.0981803jes,9.070000,313.150000,2.0,LiPF6,mol/kg,w,2.271179,0.0,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# mass_salt = molar_weights_salts[salt] * mass_solvent / ( (1 / salt_molality) - (molar_weights_salts * (1 kg / 1000 g)) )
# dimensional analysis: [kg] = (g / mol * (kg / g)) * (kg) / ((kg / mol) - ((g / mol) * (kg / g)) = ( (kg / mol) * kg) / (kg / mol)

# mol_salt = mass_salt / (molar_weights_salts[salt] * (1 kg / 1000 g) )
# dimensional analysis: [mol] = kg / (kg / mol)
molality_col_ind = salt_molality.columns.get_loc('c')
salt_name_col_ind = salt_molality.columns.get_loc('salt')
salt_name_col = salt_molality.iloc[:, salt_name_col_ind]
for i in range(len(salt_name_col)):
  salt_name = salt_name_col.iloc[i]
  MW = molar_weights_salts[salt_name] * (1 / 1000) # molecular weight converted to kg / mol: (g / mol) * (1 kg / 1000 g)
  c = salt_molality.iloc[i, molality_col_ind] # molality

  mass_salt = MW * mass_solvent / ((1 / c) - MW ) # kg = ( (kg / mol) * kg) / ( (kg / mol) - (kg / mol) )
  mol_salt = mass_salt / MW # mol = kg / (kg / mol)
  salt_molality.iloc[i, molality_col_ind] = mol_salt # salt mols reside in the molality ('c' when 'c units' = mol/kg) column

In [ ]:
salt_molality

,doi,k,T,c,salt,c units,solvent ratio type,EC,PC,DMC,...,Ethylbenzene,Ethylmonoglyme,Benzene,g-Butyrolactone,Cumene,Propylsulfone,Pseudocumeme,TEOS,m-Xylene,o-Xylene
0,10.1007/s10008-002-0300-9,0.160643,333.235034,1.17912,LiPF6,mol/kg,w,3.747445,0.0,3.66341,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10.1007/s10008-002-0300-9,0.150856,322.764924,1.17912,LiPF6,mol/kg,w,3.747445,0.0,3.66341,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10.1007/s10008-002-0300-9,0.141666,313.801722,1.17912,LiPF6,mol/kg,w,3.747445,0.0,3.66341,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,10.1007/s10008-002-0300-9,0.132277,303.681806,1.17912,LiPF6,mol/kg,w,3.747445,0.0,3.66341,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,10.1007/s10008-002-0300-9,0.119348,293.049793,1.17912,LiPF6,mol/kg,w,3.747445,0.0,3.66341,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12190,10.1149/2.0981803jes,13.930000,313.150000,2.87282,LiPF6,mol/kg,w,3.406768,0.0,7.77087,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12191,10.1149/2.0981803jes,6.360000,313.150000,2.87282,LiPF6,mol/kg,w,0.000000,0.0,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12192,10.1149/2.0981803jes,8.690000,313.150000,2.87282,LiPF6,mol/kg,w,1.135589,0.0,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12193,10.1149/2.0981803jes,9.070000,313.150000,2.87282,LiPF6,mol/kg,w,2.271179,0.0,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
copied_dataset.loc[copied_dataset['c units'] == 'mol/kg'] = salt_molality
copied_dataset.loc[copied_dataset['c units'] == 'mol/kg']

,doi,k,T,c,salt,c units,solvent ratio type,EC,PC,DMC,...,Ethylbenzene,Ethylmonoglyme,Benzene,g-Butyrolactone,Cumene,Propylsulfone,Pseudocumeme,TEOS,m-Xylene,o-Xylene
0,10.1007/s10008-002-0300-9,0.160643,333.235034,1.17912,LiPF6,mol/kg,w,3.747445,0.0,3.66341,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10.1007/s10008-002-0300-9,0.150856,322.764924,1.17912,LiPF6,mol/kg,w,3.747445,0.0,3.66341,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10.1007/s10008-002-0300-9,0.141666,313.801722,1.17912,LiPF6,mol/kg,w,3.747445,0.0,3.66341,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,10.1007/s10008-002-0300-9,0.132277,303.681806,1.17912,LiPF6,mol/kg,w,3.747445,0.0,3.66341,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,10.1007/s10008-002-0300-9,0.119348,293.049793,1.17912,LiPF6,mol/kg,w,3.747445,0.0,3.66341,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12190,10.1149/2.0981803jes,13.930000,313.150000,2.87282,LiPF6,mol/kg,w,3.406768,0.0,7.77087,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12191,10.1149/2.0981803jes,6.360000,313.150000,2.87282,LiPF6,mol/kg,w,0.000000,0.0,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12192,10.1149/2.0981803jes,8.690000,313.150000,2.87282,LiPF6,mol/kg,w,1.135589,0.0,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12193,10.1149/2.0981803jes,9.070000,313.150000,2.87282,LiPF6,mol/kg,w,2.271179,0.0,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Molarity conversion

In [ ]:
salt_molarity

,doi,k,T,c,salt,c units,solvent ratio type,EC,PC,DMC,...,Ethylbenzene,Ethylmonoglyme,Benzene,g-Butyrolactone,Cumene,Propylsulfone,Pseudocumeme,TEOS,m-Xylene,o-Xylene
23,10.1016/j.electacta.2008.04.023,4.534451,298.15,0.201932,LiPF6,mol/l,w,3.406768,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24,10.1016/j.electacta.2008.04.023,7.802607,298.15,0.498683,LiPF6,mol/l,w,3.406768,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25,10.1016/j.electacta.2008.04.023,9.348231,298.15,0.798946,LiPF6,mol/l,w,3.406768,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26,10.1016/j.electacta.2008.04.023,9.506518,298.15,1.000878,LiPF6,mol/l,w,3.406768,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
27,10.1016/j.electacta.2008.04.023,9.217877,298.15,1.201054,LiPF6,mol/l,w,3.406768,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13820,10.1016/0378-7753(91)80004-H,8.510000,333.15,1.000000,LiBF4,mol/l,v,5.938692,4.673186,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13821,10.1016/0378-7753(91)80004-H,2.530000,333.15,0.500000,LiBF4,mol/l,v,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13822,10.1016/0378-7753(91)80004-H,4.290000,353.15,1.000000,LiBF4,mol/l,v,1.960933,1.543068,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13823,10.1016/0378-7753(91)80004-H,10.790000,353.15,1.000000,LiBF4,mol/l,v,5.938692,4.673186,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# mass_salt = (molar_weights_salts[salt] * (1 kg / 1000 g) * densities_salts[salt] * salt_molarity * total_solvent_volume) / ( ( density_salts[salt]) - (salt_molarity * molar_weights_salts[salt] * (1 kg / 1000 g)) )
# dimensional analysis: kg = ( (g / mol * (1 kg / 1000 g)) * ( (g / mL) * (1 kg / 1000 g) * (1000 mL / 1 L)) * (mol / L) * L ) / ( ( ( (g / mL) * (1 kg / 1000 g) * (1000 mL / 1 L) ) -  ( (mol / L) * ( (g / mol) * (1 kg / 1000 g) ) ) ).
# dimensional analysis [simplified]: kg = ( (kg / mol ) * ( (kg / L)) * (mol / L) * L ) / ( (kg / L) - ( (mol / L) * (kg / mol) ) ) = (kg * (kg / L)) / (kg / L)
# mol_salt = mass_salt / molar_weights_salts[salt]

# convert everything to kg and L for ease
# densities_salts[salt] = g / mL * (1 kg / 1000 g) * (1000 mL / 1 L) = kg / L
# molar_weights_salts[salt] = g / mol * (1 kg / 1000 g) = kg / mol


molarity_col_ind = salt_molarity.columns.get_loc('c')
salt_name_col_ind = salt_molarity.columns.get_loc('salt')
solv_rat_type_col_ind = salt_molarity.columns.get_loc('solvent ratio type')

solv_rat_type_col = salt_molarity.iloc[:, solv_rat_type_col_ind]
salt_name_col = salt_molarity.iloc[:, salt_name_col_ind]

#og_data_molarity = dataset.loc[dataset['c units'] == 'mol/l']
#extra_dataset_molarity = extra_dataset.loc[dataset['c units'] == 'mol/l']

volumes = transform_mole_to_liter(salt_molarity,
                                  molar_weights,
                                  densities)
for i in range(len(salt_name_col)):
  salt_name = salt_name_col.iloc[i]
  salt_name = salt_name.strip()
  volume_solvent = volumes.iloc[i] # in L
  MW = molar_weights_salts[salt_name] * (1 / 1000) # molecular weight converted to kg / mol
  M = salt_molarity.iloc[i, molarity_col_ind] # molarity in [mol / L]

  mass_salt = (MW * densities_salts[salt_name] * M * volume_solvent) / ( densities_salts[salt_name] - (M * MW) ) # kg
  mol_salt = mass_salt / MW # mol
  salt_molarity.iloc[i, molarity_col_ind] = mol_salt # salt mols reside in the molarity ('c' when 'c units' = mol / L) column

In [ ]:
salt_molarity

,doi,k,T,c,salt,c units,solvent ratio type,EC,PC,DMC,...,Ethylbenzene,Ethylmonoglyme,Benzene,g-Butyrolactone,Cumene,Propylsulfone,Pseudocumeme,TEOS,m-Xylene,o-Xylene
23,10.1016/j.electacta.2008.04.023,4.534451,298.15,0.204780,LiPF6,mol/l,w,3.406768,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24,10.1016/j.electacta.2008.04.023,7.802607,298.15,0.513966,LiPF6,mol/l,w,3.406768,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25,10.1016/j.electacta.2008.04.023,9.348231,298.15,0.837246,LiPF6,mol/l,w,3.406768,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26,10.1016/j.electacta.2008.04.023,9.506518,298.15,1.060828,LiPF6,mol/l,w,3.406768,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
27,10.1016/j.electacta.2008.04.023,9.217877,298.15,1.287559,LiPF6,mol/l,w,3.406768,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13820,10.1016/0378-7753(91)80004-H,8.510000,333.15,0.889660,LiBF4,mol/l,v,5.938692,4.673186,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13821,10.1016/0378-7753(91)80004-H,2.530000,333.15,0.471158,LiBF4,mol/l,v,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13822,10.1016/0378-7753(91)80004-H,4.290000,353.15,1.175048,LiBF4,mol/l,v,1.960933,1.543068,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13823,10.1016/0378-7753(91)80004-H,10.790000,353.15,0.889660,LiBF4,mol/l,v,5.938692,4.673186,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
copied_dataset.loc[copied_dataset['c units'] == 'mol/l'] = salt_molarity
copied_dataset.loc[copied_dataset['c units'] == 'mol/l']

,doi,k,T,c,salt,c units,solvent ratio type,EC,PC,DMC,...,Ethylbenzene,Ethylmonoglyme,Benzene,g-Butyrolactone,Cumene,Propylsulfone,Pseudocumeme,TEOS,m-Xylene,o-Xylene
23,10.1016/j.electacta.2008.04.023,4.534451,298.15,0.204780,LiPF6,mol/l,w,3.406768,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24,10.1016/j.electacta.2008.04.023,7.802607,298.15,0.513966,LiPF6,mol/l,w,3.406768,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25,10.1016/j.electacta.2008.04.023,9.348231,298.15,0.837246,LiPF6,mol/l,w,3.406768,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26,10.1016/j.electacta.2008.04.023,9.506518,298.15,1.060828,LiPF6,mol/l,w,3.406768,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
27,10.1016/j.electacta.2008.04.023,9.217877,298.15,1.287559,LiPF6,mol/l,w,3.406768,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13820,10.1016/0378-7753(91)80004-H,8.510000,333.15,0.889660,LiBF4,mol/l,v,5.938692,4.673186,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13821,10.1016/0378-7753(91)80004-H,2.530000,333.15,0.471158,LiBF4,mol/l,v,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13822,10.1016/0378-7753(91)80004-H,4.290000,353.15,1.175048,LiBF4,mol/l,v,1.960933,1.543068,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13823,10.1016/0378-7753(91)80004-H,10.790000,353.15,0.889660,LiBF4,mol/l,v,5.938692,4.673186,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
copied_dataset

,doi,k,T,c,salt,c units,solvent ratio type,EC,PC,DMC,...,Ethylbenzene,Ethylmonoglyme,Benzene,g-Butyrolactone,Cumene,Propylsulfone,Pseudocumeme,TEOS,m-Xylene,o-Xylene
0,10.1007/s10008-002-0300-9,0.160643,333.235034,1.179120,LiPF6,mol/kg,w,3.747445,0.000000,3.66341,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10.1007/s10008-002-0300-9,0.150856,322.764924,1.179120,LiPF6,mol/kg,w,3.747445,0.000000,3.66341,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10.1007/s10008-002-0300-9,0.141666,313.801722,1.179120,LiPF6,mol/kg,w,3.747445,0.000000,3.66341,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,10.1007/s10008-002-0300-9,0.132277,303.681806,1.179120,LiPF6,mol/kg,w,3.747445,0.000000,3.66341,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,10.1007/s10008-002-0300-9,0.119348,293.049793,1.179120,LiPF6,mol/kg,w,3.747445,0.000000,3.66341,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13820,10.1016/0378-7753(91)80004-H,8.510000,333.150000,0.889660,LiBF4,mol/l,v,5.938692,4.673186,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13821,10.1016/0378-7753(91)80004-H,2.530000,333.150000,0.471158,LiBF4,mol/l,v,0.000000,0.000000,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13822,10.1016/0378-7753(91)80004-H,4.290000,353.150000,1.175048,LiBF4,mol/l,v,1.960933,1.543068,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13823,10.1016/0378-7753(91)80004-H,10.790000,353.150000,0.889660,LiBF4,mol/l,v,5.938692,4.673186,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Export to csv

In [ ]:
copied_dataset_drop = copied_dataset.reset_index(drop=True)

In [ ]:
copied_dataset_drop.to_csv('Pre-processed Data.csv')

In [ ]:
test = pd.read_csv('Pre-processed Data.csv')
test
test = test.drop(columns=['Unnamed: 0'])
test

,doi,k,T,c,salt,c units,solvent ratio type,EC,PC,DMC,...,Ethylbenzene,Ethylmonoglyme,Benzene,g-Butyrolactone,Cumene,Propylsulfone,Pseudocumeme,TEOS,m-Xylene,o-Xylene
0,10.1007/s10008-002-0300-9,0.160643,333.235034,1.179120,LiPF6,mol/kg,w,3.747445,0.000000,3.66341,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10.1007/s10008-002-0300-9,0.150856,322.764924,1.179120,LiPF6,mol/kg,w,3.747445,0.000000,3.66341,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10.1007/s10008-002-0300-9,0.141666,313.801722,1.179120,LiPF6,mol/kg,w,3.747445,0.000000,3.66341,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,10.1007/s10008-002-0300-9,0.132277,303.681806,1.179120,LiPF6,mol/kg,w,3.747445,0.000000,3.66341,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,10.1007/s10008-002-0300-9,0.119348,293.049793,1.179120,LiPF6,mol/kg,w,3.747445,0.000000,3.66341,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13297,10.1016/0378-7753(91)80004-H,8.510000,333.150000,0.889660,LiBF4,mol/l,v,5.938692,4.673186,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13298,10.1016/0378-7753(91)80004-H,2.530000,333.150000,0.471158,LiBF4,mol/l,v,0.000000,0.000000,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13299,10.1016/0378-7753(91)80004-H,4.290000,353.150000,1.175048,LiBF4,mol/l,v,1.960933,1.543068,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13300,10.1016/0378-7753(91)80004-H,10.790000,353.150000,0.889660,LiBF4,mol/l,v,5.938692,4.673186,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
